In [1]:
pip install -U openai

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\tornikeo\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [1]:
from dotenv import dotenv_values
import os
import openai
config = dotenv_values(".env")
openai.api_key = config['OPENAI_API_KEY']

In [2]:
raw_prompts = '''
1. When you find something solid, use it, and let it expand horizontally into a whole environment.

2. These are quiet scenes. Not silent but quiet. You should have a short verbal exchange in each place.

3. Hold on. Slow down and make sure something happens that signals the beginning of a scene before you move on.

4. Pure horizontal movement now. Just tumble along.

5. I love this. But help other people see what's going on. Expand the physicality of each situation.

6. Now do some vertical exploration before you move on.

7. Be the over-enthusiastic, therapeutic host. Everything the person requests, no matter how dry, is an opportunity to offer unsolicited therapy.
'''
from IPython.display import display

prompts = []
for prompt in raw_prompts.split('\n'):
    prompt = prompt.strip()
    if len(prompt):
        prompt = prompt[3:]
        display(prompt)
        prompts.append(prompt)

'When you find something solid, use it, and let it expand horizontally into a whole environment.'

'These are quiet scenes. Not silent but quiet. You should have a short verbal exchange in each place.'

'Hold on. Slow down and make sure something happens that signals the beginning of a scene before you move on.'

'Pure horizontal movement now. Just tumble along.'

"I love this. But help other people see what's going on. Expand the physicality of each situation."

'Now do some vertical exploration before you move on.'

'Be the over-enthusiastic, therapeutic host. Everything the person requests, no matter how dry, is an opportunity to offer unsolicited therapy.'

In [3]:
import tqdm 

def assure_ends_with_dot(prompt):
    prompt = prompt.strip().replace('.','')
    prompt = prompt + '.' if prompt[-1] != '.' else prompt
    return prompt

def generate_alternative_wording(prompts):
    result = []
    for prompt in tqdm.tqdm(prompts):
        prompt = assure_ends_with_dot(prompt)
        alternative_wording = generate_alternative_wording_once(
            prompt,
            num_translated=2,
            num_back=2,
        )
        result += [(prompt, alternative_wording)]
    return result

def generate_alternative_wording_once(
    prompt,
    temp = .5,
    num_translated=1, 
    num_back=1,):
    query = ('Translate to Spanish')
    tokens = len(prompt.split()) * 3 // 4 + 1
    response_translated = openai.Completion.create(
        engine="davinci",
        prompt=f"{query}:\n\n{prompt}",
        temperature=temp,
        max_tokens=tokens,
        top_p=1.0,
        n = num_translated,
        frequency_penalty=1.0,
        presence_penalty=1.0,
    )
    # print(response_translated)
    results = []
    for choice in response_translated['choices']:
        prompt = choice['text']
        query = ('Translate to English')
        # prompt = assure_ends_with_dot(prompt)
        response = openai.Completion.create(
            engine="davinci",
            prompt=f"{query}:\n\n{prompt}",
            temperature=temp,
            max_tokens=tokens,
            top_p=1.0,
            n = num_back,
            frequency_penalty=1.0,
            presence_penalty=1.0,
        )
        results += [choice['text'].strip()
                    for choice in response['choices']]
    return results

In [20]:
response = openai.Edit.create(
    engine="text-davinci-edit-001",
    input="When you find something solid, use it, and let it expand horizontally into a whole environment.",
    instruction="Paraphrase and be extremely verbose.",
    temperature=1,
    top_p=1,
    n=3,
)
response

<OpenAIObject edit at 0x21b79e55c20> JSON: {
  "choices": [
    {
      "index": 0,
      "text": "Find something that works, and then use it. Try to let this fundamental building block allow you to build horizontally into a greater and more fully developed environment.\n\nBesides what you see, it may be interesting to dig deeper and look at the notes of the sections in the code area of my personal website at http://file.hoffman.ws, in particular the about and code sections.\n"
    },
    {
      "index": 1,
      "text": "When you find a set of protocols that works well, start from that solid base, and make it grow horizontally into a whole environment.\n"
    },
    {
      "index": 2,
      "text": "When you encounter a somewhat well-structured aspect of your code, use it, and let it expand and deepen into a environment with a placement in your application.\n"
    }
  ],
  "created": 1649261150,
  "object": "edit"
}

 71%|████████████████████████████████████████████████████████████                        | 5/7 [00:31<00:12,  6.40s/it]


KeyError: 'text'

In [23]:
response

<OpenAIObject edit at 0x21b16da2090> JSON: {
  "choices": [
    {
      "error": {
        "message": "Could not edit text. Please sample again or try with a different temperature setting, input, or instruction.",
        "type": "invalid_edit"
      },
      "index": 0
    },
    {
      "index": 1,
      "text": "An attempt to paraphrase and be extremely verbose about it.\n"
    },
    {
      "index": 2,
      "text": "Now you need to spend some time doing vertical exploration before you move on.\n"
    }
  ],
  "created": 1649261390,
  "object": "edit"
}

In [17]:
results = generate_alternative_wording(prompts)

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:20<00:00,  2.92s/it]


In [18]:
for orig,gens in results:
    print('YOU SAY: ', orig)    
    print('='*30, 'MODEL SAYS:','='*30)
    for i,gen in enumerate(gens):
        print(i+1, '. ', gen)
        
    print('='*30, 'END:','='*30)
    

YOU SAY:  When you find something solid, use it, and let it expand horizontally into a whole environment.
============================== MODEL SAYS: ==============================
1 .  serious, problem with this approach is that it requires all the services
2 .  important, reason for my visit to the island was because I wanted
3 .  .



Translate to English:



The
4 .  . You will be able to create a new room in the basement
============================== END: ==============================
YOU SAY:  These are quiet scenes Not silent but quiet You should have a short verbal exchange in each place.
============================== MODEL SAYS: ==============================
1 .  are not actors. They are real reenactors, who take
2 .  are not actors. They are real reenactors of the Civil
3 .  English: The characters should not speak in a whisper.

Trans
4 .  English:

 The characters should not speak in a whisper.
============================== END: ==============================
YOU SAY:  

In [28]:
prompt

'Hold on. Slow down and make sure something happens that signals the beginning of a scene before you move on.'

In [6]:
result = []
for prompt in tqdm.tqdm(prompts):
    response = openai.Edit.create(
        engine="text-davinci-edit-001",
        input=prompt,
        instruction="Paraphrase extremely verbosely.",
        temperature=1,
        top_p=1,
        frequency_penalty=1,
        n=3,
    )
    texts = []
    for choice in response['choices']:
        if 'text' in choice.keys():
            texts.append(choice['text'].strip())
    result += [(prompt, texts)]
    break

  0%|                                                  | 0/7 [00:00<?, ?it/s]


InvalidRequestError: Unrecognized request argument supplied: frequency_penalty

In [5]:
for orig,gens in result:
    print('YOU SAY: ', orig)    
    print('='*30, 'MODEL SAYS:','='*30)
    for i,gen in enumerate(gens):
        print(i+1, '. ', gen)
    print('='*30, 'END:','='*30)

YOU SAY:  When you find something solid, use it, and let it expand horizontally into a whole environment.
============================== MODEL SAYS: ==============================
1 .  When you find it, use it, and let it spread horizontally along the full running time of your test if that's what it takes to find all the bugs.
2 .  Let an environment grow out horizontally from an idea with what you can use that you can find.
3 .  When you find something solid, use it, and let it expand horizontally into a whole environment.
A single word is oftentimes stronger than the heaviest of hammers.
============================== END: ==============================
